In [1]:
import sys
import imp
sys.path.insert(0, '../src')

In [66]:
from denoise.graph import io as graphio
imp.reload(graphio)

_, proteins_to_go = graphio.parse_go_label_file("../data/dream_files/human.golabels")
edgelist, node_list, node_map = graphio.parse_graph_file("../data/dream_files/dream_3.txt")

In [57]:
from denoise.graph import operations
from denoise.algorithms.dsd import computations
imp.reload(operations)
imp.reload(computations)

A = operations.densify(edgelist)
D = computations.compute_degree_mat(A)
X = computations.compute_X_normalized(A, D)

In [90]:
import denoise.algorithms.dsd.denoise as denoise
imp.reload(denoise)

# Useful for adding edges as well (though more computationally
# expensive
all_edges = denoise.predict_links(X) 

# Rank the edges in the edgelist according to the embedding
# and remove bottom 10 percent of edges
ranked_edges = denoise.rank_edges(edgelist, X)
bad_edges = ranked_edges[-int(len(ranked_edges) * 0.5):]

SyntaxError: unexpected EOF while parsing (<ipython-input-90-725d51d3cf32>, line 18)

In [97]:
A_removed_edges = A.copy()
for (u, v, _, _) in bad_edges:
    A_removed_edges[u, v] = 0

A_added_edges = A.copy()
num_added, counter = 0, 0
while num_added < len(edgelist) * 0.10:
    (u, v), _ = all_edges[counter]
    if A_added_edges[u, v] == 0:
        A_added_edges[u, v] = 1
        num_added += 1
    counter += 1

In [98]:
D_added_edges = computations.compute_degree_mat(A_added_edges)
X_added_edges = computations.compute_X_normalized(A_added_edges, D_added_edges)
D_removed_edges = computations.compute_degree_mat(A_removed_edges)
X_removed_edges = computations.compute_X_normalized(A_removed_edges, D_removed_edges)

In [104]:
from denoise.algorithms.dsd import computations
imp.reload(computations)

S = computations.compute_sim_matrix(X, params={"rbf": 1})
S_removed_edges = computations.compute_sim_matrix(X_removed_edges, params={"rbf": 1})
S_added_edges = computations.compute_sim_matrix(X_added_edges, params={"rbf": 1})

In [105]:
from denoise import scoring
from denoise import predict
imp.reload(predict)
imp.reload(scoring)

n, _ = A.shape
labels = {i: proteins_to_go[node_list[i]] for i in range(n)
          if node_list[i] in proteins_to_go}

def create_predictor(similarity_matrix):
    def predictor(training_labels):
        tlabels_f = lambda i: (training_labels[i] if i in training_labels else [])
        return predict.wmv(similarity_matrix, tlabels_f)
    return predictor
 
# A_scores = scoring.kfoldcv(5, labels, create_predictor(A))
# A_added_edges_scores = scoring.kfoldcv(5, labels, create_predictor(A_added_edges))
# A_removed_edges_scores = scoring.kfoldcv(5, labels, create_predictor(A_removed_edges))
S_scores = scoring.kfoldcv(5, labels, create_predictor(S))
S_removed_edges_scores = scoring.kfoldcv(5, labels, create_predictor(S_removed_edges))
S_added_edges_scores = scoring.kfoldcv(5, labels, create_predictor(S_added_edges))

print(f"The scores for running 10-fold cv using WMV on the original network are: {A_scores}")
print(f"The scores for running 10-fold cv using WMV on the denoised original network are: {A_removed_edges_scores}")
print(f"The scores for running 10-fold cv using WMV on the embedded network are: {S_scores}")
print(f"The scores for running 10-fold cv using WMV on the denoised embedded network are: {S_removed_edges_scores}")

The scores for running 10-fold cv using WMV on the original network are: [0.13961407491486946, 0.1362088535754824, 0.11350737797956867, 0.09988649262202043, 0.09648127128263337]
The scores for running 10-fold cv using WMV on the denoised original network are: [0.1362088535754824, 0.13280363223609534, 0.10783200908059024, 0.08740068104426787, 0.05788876276958002]
The scores for running 10-fold cv using WMV on the embedded network are: [0.19069239500567536, 0.1532349602724177, 0.11464245175936436, 0.09761634506242906, 0.09761634506242906]
The scores for running 10-fold cv using WMV on the denoised embedded network are: [0.19069239500567536, 0.15209988649262202, 0.11237230419977298, 0.09875141884222474, 0.09761634506242906]


In [106]:
print(np.mean(A_scores) * 100)
print(np.mean(A_added_edges_scores) * 100)
print(np.mean(A_removed_edges_scores) * 100)
print(np.mean(S_scores) * 100)
print(np.mean(S_removed_edges_scores) * 100)
print(np.mean(S_added_edges_scores) * 100)

11.713961407491487
11.713961407491487
10.442678774120317
13.076049943246309
13.030646992054482
13.030646992054484
